# Gets all the edits for wikipedia pages

In [1]:
import requests
import stringcase
import pandas as pd
from pathlib import Path

Prepares url and parameters for api call

In [2]:
session = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

PARAMS = {
    "action": "query",
    "format": "json",
    "prop": "revisions",
    "rvlimit":"max",
    "rvprop": "timestamp|user|userid|sha1|flags",
}

Creates csv file for edits for each pages

In [3]:
data = []
loop = True
path = "./../DATA/Edits"
df = pd.read_csv("./../DATA/pages/pages_filtered.csv")

Path(path).mkdir(parents=True, exist_ok=True)

for index, row in df.iterrows():
    while loop:
        PARAMS['pageids'] = row['page_id']
        query_data = session.get(url=URL, params=PARAMS).json()
        
        if 'continue' in query_data and 'rvcontinue' in query_data['continue']:
            PARAMS['rvcontinue'] = query_data['continue']['rvcontinue']
        else: loop = False
            
        for revision in query_data['query']['pages']['{}'.format(row['page_id'])]['revisions']:
                sha1 =  revision['sha1'] if 'sha1' in revision else revision['sha1hidden']
                user = revision['user'] if 'user' in revision else revision['userhidden']
                
                data.append([sha1, user, revision['timestamp']])
           
    filtered_df = pd.DataFrame(data, columns=['sha1', 'user', 'timestamp'])
    file_name = stringcase.snakecase( '{} {}'.format(row['title'], row['page_id'])).replace('/', '_')
    
    filtered_df.to_csv("{}/{}.csv".format(path, file_name),encoding = "utf-8", index=False, header=True)

    if 'rvcontinue' in PARAMS:
        del PARAMS['rvcontinue']
    
    loop = True
    data.clear()